In [61]:
import pandas as pd
import matplotlib.pyplot as plt

### importação de dados ###

In [62]:
df = pd.read_csv('zap_imoveis.csv', index_col='id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3944 entries, 0 to 3943
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   condominio   3944 non-null   float64
 1   anunciante   3944 non-null   object 
 2   num_suites   3944 non-null   float64
 3   num_vagas    3944 non-null   float64
 4   num_quartos  3944 non-null   float64
 5   area         3944 non-null   float64
 6   rua          2932 non-null   object 
 7   regiao       3944 non-null   object 
 8   descricao    3764 non-null   object 
 9   tipo_imovel  3944 non-null   object 
 10  localidade   3944 non-null   object 
 11  bairro       3944 non-null   object 
 12  preco        3944 non-null   int64  
dtypes: float64(5), int64(1), object(7)
memory usage: 431.4+ KB


### limpeza da base ###

In [63]:
df = df.drop(['anunciante','descricao','localidade','rua','regiao','bairro'], axis = 1)
df = df.query("tipo_imovel == 'Apartamento' or tipo_imovel == 'Casa' or tipo_imovel == 'Casa de Condomínio'")
df.head().T

id,0,1,2,3,4
condominio,0.0,0.0,18.0,230.0,680.0
num_suites,2.0,0.0,0.0,0.0,1.0
num_vagas,2.0,0.0,1.0,1.0,1.0
num_quartos,3.0,0.0,2.0,2.0,3.0
area,150.0,360.0,45.0,55.0,147.0
tipo_imovel,Casa,Casa,Casa,Casa,Casa
preco,570000,200000,170000,250000,600000


In [64]:
df['tipo_imovel'].value_counts()


Apartamento           1623
Casa                  1502
Casa de Condomínio     150
Name: tipo_imovel, dtype: int64

### definição de dados de treino e teste ###

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('tipo_imovel',axis=1), df['tipo_imovel'], test_size=0.3)

### Definição do número de vizinhos no classificador ###

In [66]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

### aplicando dados de treino ao knn ###

In [67]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

### aplicando knn aos dados de teste ###

In [68]:
resultado = knn.predict(X_test)

### criando matriz de confusão ###

In [69]:
print (pd.crosstab(y_test,resultado, rownames=['Real'], colnames=['Predicted'], margins=True))

Predicted           Apartamento  Casa  Casa de Condomínio  All
Real                                                          
Apartamento                 387    99                   6  492
Casa                        108   326                   2  436
Casa de Condomínio           33    17                   5   55
All                         528   442                  13  983


### gerando as métricas de classificação ###

In [70]:
from sklearn import metrics
print(metrics.classification_report(y_test,resultado))

                    precision    recall  f1-score   support

       Apartamento       0.73      0.79      0.76       492
              Casa       0.74      0.75      0.74       436
Casa de Condomínio       0.38      0.09      0.15        55

          accuracy                           0.73       983
         macro avg       0.62      0.54      0.55       983
      weighted avg       0.72      0.73      0.72       983



### verificando ajustes para o modelo ###

In [71]:
from sklearn.model_selection import GridSearchCV


### adicionando a quantidade de valores para k (número de vizinhos) a um dicionário ###

In [72]:
k_list = list(range(1,31))
parametros = dict(n_neighbors=k_list)


### passando nosso modelo, os valores para k, dobras pra validação cruzada e o scoring para o objeto GridSearch ###

In [73]:
grid = GridSearchCV(knn, parametros, cv=5, scoring='accuracy')


### treinando o objeto ###

In [74]:
grid.fit(df.drop('tipo_imovel',axis=1), df['tipo_imovel'])

GridSearchCV(cv=5, estimator=KNeighborsClassifier(n_neighbors=3),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30]},
             scoring='accuracy')

### descobrindo o parâmetro com melhor acurácia ###

In [75]:
print("Melhor parâmetro {} com acurácia de {} ".format(grid.best_params_,grid.best_score_))

Melhor parâmetro {'n_neighbors': 3} com acurácia de 0.7282442748091602 
